In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 28.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [5]:
from datasets import load_dataset

In [6]:
dataset=load_dataset('cnn_dailymail','1.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
print(dataset['train'])

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})


In [8]:
def preprocess_text(text):

# remove stop words and digits

    doc = nlp(text)

    tokens = [token.text.lower() for token in doc if not token.is_stop and token.is_alpha]

    preprocessed_text = " ".join(tokens)

    return preprocessed_text


In [9]:
preprocessed_corpus=[]
corpus=[]

for i in range(100): # only taken 100 examples can take entire dataset which has 287113 records
  text=preprocess_text(dataset['train'][i]['highlights'])
  # text=dataset['train'][i]['highlights']
  doc=nlp(text)
  preprocessed_corpus.extend([chunk.text for chunk in doc.noun_chunks])

print(preprocessed_corpus)

['harry potter star daniel radcliffe', 'monday young actor', 'fritter cash away radcliffe earnings potter films', 'trust fund', 'mentally ill inmates', 'miami', 'forgotten floor judge steven leifman', 'avoidable felonies cnn tours facility patient', 'son president leifman', 'new thought', 'die driver', 'man', 'pickup truck', 'half cut face driver', 'free fall minnesota bridge', 'small polyps', 'procedure worrisome spokesman', 'president', 'powers', 'vice president bush', 'routine colonoscopy camp', 'new nfl chief atlanta falcons owner critical michael vick conduct nfl', 'falcons', 'quarterback', 'vick', 'funding dogfighting operation', 'gamble vick federal court', 'monday', 'future nfl', 'parents beam pride', 'support mom', 'happy know burn center', 'treatment', 'reconstructive surgeries', 'dad', 'youssif', 'aid workers violence', 'cost living drive women prostitution group working raise awareness problem iraq political leaders', 'cnn', 'children', 'children woman', 'medina caracas', '

In [10]:
tfidf = TfidfVectorizer()
context_matrix = tfidf.fit_transform(preprocessed_corpus)


tfidf.get_feature_names_out()

array(['abandoned', 'abortion', 'abuse', ..., 'zealand', 'zoe',
       'zvonareva'], dtype=object)

In [11]:
def cosine_similarity_between_noun_phrases(np1, np2):
    np1_index = preprocessed_corpus.index(np1)
    np2_index = preprocessed_corpus.index(np2)
    cosine_sim = cosine_similarity(context_matrix[np1_index], context_matrix[np2_index])
    return cosine_sim[0][0]

In [12]:
np1 = "harry potter star daniel radcliffe"
np2 = "mentally ill inmates"
similarity = cosine_similarity_between_noun_phrases(np1, np2)
print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

np1 = "harry potter star daniel radcliffe"
np2 =  "fritter cash away radcliffe earnings potter films"
similarity = cosine_similarity_between_noun_phrases(np1, np2)
print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

# np1 = "Harry Potter star Daniel Radcliffe"
# np2 = "Mentally ill inmates"
# similarity = cosine_similarity_between_noun_phrases(np1, np2)
# print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

# np1 = "Harry Potter star Daniel Radcliffe"
# np2 =  "Young actor"
# similarity = cosine_similarity_between_noun_phrases(np1, np2)
# print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

# np1 = "Harry Potter star Daniel Radcliffe"
# np2 =  "Radcliffe's earnings"
# similarity = cosine_similarity_between_noun_phrases(np1, np2)
# print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

# np1 = "Harry Potter star Daniel Radcliffe"
# np2 =  "his cash"
# similarity = cosine_similarity_between_noun_phrases(np1, np2)
# print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

Cosine similarity between 'harry potter star daniel radcliffe' and 'mentally ill inmates': 0.0
Cosine similarity between 'harry potter star daniel radcliffe' and 'fritter cash away radcliffe earnings potter films': 0.32140297875584767


## **LAB 4**

In [13]:
from gensim.models import Word2Vec
from collections import Counter

In [14]:
noun_phrases=preprocessed_corpus

In [15]:
sentences = [sentence.split() for sentence in noun_phrases]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)

In [19]:
phrases = [phrase for noun_phrase in sentences for phrase in noun_phrase]
phrase_counter = Counter(phrases)
top_k_phrases = phrase_counter.most_common(100)
print("Top {} frequent phrases:".format(10))
for phrase, freq in top_k_phrases:
    print("{}: {}".format(phrase, freq))
print(top_k_phrases)

Top 10 frequent phrases:
new: 25
president: 15
bush: 11
house: 11
man: 9
children: 9
iraq: 8
white: 7
police: 7
year: 7
iraqi: 7
people: 6
company: 6
world: 6
cnn: 5
group: 5
charges: 5
students: 5
cave: 5
monday: 4
nfl: 4
court: 4
aid: 4
leaders: 4
foreign: 4
number: 4
states: 4
leader: 4
france: 4
war: 4
embassy: 4
officials: 4
second: 4
museum: 4
south: 4
school: 4
refugee: 4
official: 4
fire: 4
star: 3
facility: 3
face: 3
routine: 3
colonoscopy: 3
chief: 3
owner: 3
vick: 3
workers: 3
military: 3
snow: 3
weapon: 3
home: 3
experts: 3
veterans: 3
terrorists: 3
debate: 3
london: 3
july: 3
brazilian: 3
league: 3
saturday: 3
run: 3
men: 3
life: 3
islamic: 3
look: 3
australia: 3
uk: 3
robbery: 3
conspiracy: 3
related: 3
countries: 3
human: 3
striker: 3
principal: 3
employees: 3
diana: 3
best: 3
earth: 3
french: 3
government: 3
minister: 3
student: 3
dead: 3
gupta: 3
questions: 3
movie: 3
boy: 3
united: 3
old: 3
coalition: 3
mayor: 3
homes: 3
red: 3
families: 3
members: 3
soldier: 3
safe: 

In [20]:
phrases = ['diana','french','uk','nfl','bush','australia']

for i in phrases:
  similar_phrase = word2vec_model.wv.most_similar(i, topn=10)
  print()
  print("Similar Phrases to: " + i)
  print()
  for phrase, similarity in similar_phrase:
    print(f"{phrase}: {similarity}")


Similar Phrases to: diana

armenians: 0.3251366913318634
cut: 0.29727745056152344
medical: 0.2968105971813202
tragedy: 0.28530165553092957
disputes: 0.2830711901187897
bangladesh: 0.27814507484436035
shooting: 0.2685549557209015
burglary: 0.25135287642478943
sports: 0.24896343052387238
patil: 0.24762161076068878

Similar Phrases to: french

mate: 0.2877671420574188
spitting: 0.2783522307872772
cash: 0.27049627900123596
lindsay: 0.25324997305870056
ark: 0.2510293126106262
milan: 0.2495686113834381
suicide: 0.24943898618221283
northeastern: 0.24865631759166718
retired: 0.23707659542560577
work: 0.23688121140003204

Similar Phrases to: uk

italy: 0.29255327582359314
area: 0.28948646783828735
juventus: 0.28791511058807373
cars: 0.28234586119651794
round: 0.2790181636810303
injuries: 0.26731470227241516
gamble: 0.25910866260528564
head: 0.2562035024166107
blast: 0.25166356563568115
bodies: 0.24979105591773987

Similar Phrases to: nfl

fighters: 0.36239054799079895
election: 0.2879338860511